In [1]:
!wget https://github.com/jcpeterson/percept2vec/blob/master/animals.zip?raw=true
!unzip animals.zip?raw=true
!wget https://github.com/jcpeterson/percept2vec/blob/master/turkResults_CogSci2016.mat?raw=true
!mv turkResults_CogSci2016.mat\?raw\=true hum.mat
import scipy.io
hum = scipy.io.loadmat('hum.mat')
humMat = hum['simMatrix']


--2021-05-04 16:27:53--  https://github.com/jcpeterson/percept2vec/blob/master/animals.zip?raw=true
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/jcpeterson/percept2vec/raw/master/animals.zip [following]
--2021-05-04 16:27:54--  https://github.com/jcpeterson/percept2vec/raw/master/animals.zip
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jcpeterson/percept2vec/master/animals.zip [following]
--2021-05-04 16:27:54--  https://raw.githubusercontent.com/jcpeterson/percept2vec/master/animals.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting resp

In [51]:
humMat

array([[10. ,  3.9,  0.6, ...,  4.9,  0.1,  0.3],
       [ 3.9, 10. ,  0.2, ...,  5. ,  0.9,  0.3],
       [ 0.6,  0.2, 10. , ...,  0.1,  1.1,  3.2],
       ...,
       [ 4.9,  5. ,  0.1, ..., 10. ,  0.3,  0.6],
       [ 0.1,  0.9,  1.1, ...,  0.3, 10. ,  0.3],
       [ 0.3,  0.3,  3.2, ...,  0.6,  0.3, 10. ]])

In [2]:
import torch
import torch.nn as nn
model = torch.hub.load('pytorch/vision:v0.6.0', 'alexnet', pretrained=True)

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /home/dawooood/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth" to /home/dawooood/.cache/torch/hub/checkpoints/alexnet-owt-4df8aa71.pth


In [3]:
model.classifier = nn.Sequential(*list(model.classifier.children())[:-1])

In [4]:
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [8]:
import glob
import re
from tensorflow.keras.preprocessing import image
from PIL import Image
import numpy as np

data = {}
name=[]
path = "images/*.png"

for file in glob.glob(path):
    temp1=image.load_img(file)
    temp2=re.findall(r'\d+', file)
    name.append(int(temp2[0]))
    data.update({int(temp2[0]) : temp1})

print(len(name))

mapping = {}


120


In [17]:
from torchvision import transforms

preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(227),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [18]:
for i in range(120):
  input_tensor = preprocess(data[name[i]])
  input_batch = input_tensor.unsqueeze(0)
  feature = model(input_batch)
  mapping.update({name[i] : feature})


In [24]:
len(mapping.keys())

120

In [30]:
mapping[307].cpu().detach().numpy()

array([[0.       , 0.       , 0.       , ..., 1.9570031, 0.       ,
        0.       ]], dtype=float32)

In [32]:
name.sort()

li = []

for i in range(120):
  li.append(mapping[name[i]].cpu().detach().numpy())

F = np.asarray(li)
F = np.reshape(F, (120,4096))
print(F.shape)

Mat = F.dot(F.transpose())
OrigSimMat = Mat
Mat = np.reshape(Mat, (14400))
print(Mat.shape)

(120, 4096)
(14400,)


In [38]:
OrigSimMat

array([[3088.7007 ,  877.708  ,  317.09296, ...,  910.2732 ,  569.53516,
         584.09814],
       [ 877.708  , 4287.103  ,  559.81195, ...,  744.2179 ,  335.8089 ,
         468.08844],
       [ 317.09296,  559.81195, 5449.1006 , ...,  318.2934 , 1194.7544 ,
         522.56757],
       ...,
       [ 910.2732 ,  744.2179 ,  318.2934 , ..., 8050.0713 ,  641.00574,
        1114.5996 ],
       [ 569.53516,  335.8089 , 1194.7544 , ...,  641.00574, 7745.916  ,
         197.30289],
       [ 584.09814,  468.08844,  522.56757, ..., 1114.5996 ,  197.30289,
        3392.2656 ]], dtype=float32)

In [37]:
Mat

array([3088.7007 ,  877.708  ,  317.09296, ..., 1114.5996 ,  197.30289,
       3392.2656 ], dtype=float32)

In [23]:
humMat = np.reshape(humMat,(14400,))

cor_mat = np.corrcoef(Mat, humMat)
r = cor_mat[0,1]

r2 = r**2
print(r2)

0.3162062234907594


In [42]:
humanSimMat = humMat.reshape(120,120)
uniMat=[]               # For considering only the unique entries since it is a symmetric matrix
for i in range(120):
    for j in range(i,120):
        uniMat.append(humanSimMat[i,j])

uniMat=np.asarray(uniMat)

print(uniMat.shape)

(7260,)


In [43]:
inp = []
for i in range(120):
    for j in range(i,120):
        inp.append(F[i]*F[j])

inp = np.asarray(inp)
print(inp.shape )

inp2 = []
for i in range(120):
    for j in range(120):
        inp2.append(F[i]*F[j])

inp2 = np.asarray(inp2)
print(inp2.shape )

(7260, 4096)
(14400, 4096)


In [49]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge

reg = Ridge(solver='sag', fit_intercept=False)

parameters = {'alpha': [10,100,1000,1e4, 50000, 1e5,1e6]}

search = GridSearchCV(reg, parameters, scoring='neg_mean_squared_error', cv=6)
search.fit(inp, uniMat)

NameError: name '_private' is not defined

In [45]:
best_reg = search.best_estimator_
print(best_reg)

NameError: name 'search' is not defined

In [46]:
from sklearn.model_selection import cross_validate
a=cross_validate(best_reg,inp2,humMat,scoring="r2",cv=6)

AttributeError: module 'numpy.linalg.lapack_lite' has no attribute '_ilp64'

In [47]:
a['test_score'].mean()

NameError: name 'a' is not defined

In [26]:
PredSimMat = best_reg.predict(inp2)

cor_mat = np.corrcoef(PredSimMat, humMat)
r = cor_mat[0,1]

r2 = r**2
print(r2)

0.6656524624393979
